In [10]:
import torch
import torch.quantization
from model_training.utils.models import MobileNetV3
import torch.nn as nn
from torch.utils.data import DataLoader
from model_training.utils.data_preparation import CreateDataset, load_images_and_labels, encode_labels, create_data_augmentation
from model_training.utils.trainer import evaluate_model
from sklearn.model_selection import train_test_split
from torch.quantization import fuse_modules


model = MobileNetV3(num_labels=7)
model.load_state_dict(torch.load('models/distilled_lottery_ticket_ck.pt'))
model.eval()


/tmp/ipykernel_534119/192243618.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/distilled_lottery_ticket_ck.pt'))


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [11]:
model.to('cpu')

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [12]:
def fuse_supported_layers(model):
    for name, module in model.named_children():
        if isinstance(module, nn.Sequential):
            # Fuse Conv2d + BatchNorm2d + ReLU layers within the sequential module
            for i in range(len(module) - 1):
                if isinstance(module[i], nn.Conv2d) and isinstance(module[i+1], nn.BatchNorm2d):
                    if i+2 < len(module) and isinstance(module[i+2], nn.ReLU):
                        fuse_modules(module, [str(i), str(i+1), str(i+2)], inplace=True)
                    else:
                        fuse_modules(module, [str(i), str(i+1)], inplace=True)
        elif isinstance(module, nn.Conv2d) and isinstance(module, nn.BatchNorm2d):
            # Directly fuse Conv2d + BatchNorm2d
            fuse_modules(model, [name], inplace=True)
        else:
            # Recursively fuse the layers in child modules
            fuse_supported_layers(module)

# Apply selective layer fusion to the model
fuse_supported_layers(model)

In [13]:
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

In [14]:
torch.quantization.prepare(model, inplace=True)

/home/diogoalves/thesis/.venv/lib/python3.10/site-packages/torch/ao/quantization/observer.py:221: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(
        3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
        (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
      )
      (1): BatchNorm2d(
        16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True
        (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
      )
      (2): Hardswish(
        (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
      )
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(
            16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False
            (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
          )
          (1): BatchNorm2d(
            16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True
            (activation_post_proc

In [16]:
data = "ck"
train_path = f"data/{data}/train"
test_path = f"data/{data}/test"

X_train, y_train = load_images_and_labels(train_path)
X_test, y_test = load_images_and_labels(test_path)

X = X_train + X_test
y = y_train + y_test


X_train, X_test_and_val, y_train, y_test_and_val = train_test_split(X, y,
                                                    test_size=0.5,
                                                    random_state=42)

X_test, X_val, y_test, y_val = train_test_split(X_test_and_val,
                                                y_test_and_val,
                                                test_size=0.5,
                                                random_state=42)

label_encoder, y_train, y_val, y_test = encode_labels(y_train, y_val, y_test)

transform = create_data_augmentation()

calibration_dataset = CreateDataset(X_val, y_val, transform=transform)
test_dataset = CreateDataset(X_test, y_test, transform=transform)

calibration_loader = DataLoader(calibration_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for i, (input, target) in enumerate(calibration_loader):
    if i >= 100:
        break
    model(input)

In [17]:
quantized_model = torch.quantization.convert(model, inplace=True)

In [18]:
torch.save(quantized_model.state_dict(), 'models/mobilenet_v3_quantized.pt')

In [20]:
evaluate_model(quantized_model, test_loader, "cpu", "quantized model", label_encoder=label_encoder)

NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

Meta: registered at ../aten/src/ATen/core/MetaFallbac